<a href="https://colab.research.google.com/github/dindorios/dindorios/blob/main/Git_HUB_Reconhecimento_por_imagem_Beb%C3%AA_ou_Nao_Beb%C3%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow opencv-python

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
import os

In [ ]:
# 1. Modelo pré-treinado (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False # Congela as camadas convolucionais

In [ ]:
# 2. Adaptação para classificar bebês
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x) # 2 classes: bebê ou não bebê
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# 3. Carregamento dos dados de treinamento (você precisará preparar as imagens carregando imagens diversas de pessoas e de bebês separados por pastas)
# Este exemplo assume que você tem pastas 'bebe' e 'nao_bebe' no Colab
train_data_dir = '/content/Imagens' # Ajuste o caminho criando duas pastas dentro da pasta Imagens ou alguma de sua escolha
img_width, img_height = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 14 images belonging to 2 classes.


In [ ]:
# 4. Treinamento do modelo (opcional, se quiser ajustar o modelo)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=50) # Ajuste o número de épocas

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.5000 - loss: 0.7217
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 733ms/step - accuracy: 0.5714 - loss: 1.1428
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 724ms/step - accuracy: 1.0000 - loss: 0.0048
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0011
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 704ms/step - accuracy: 1.0000 - loss: 0.0203
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 688ms/step - accuracy: 0.9286 - loss: 0.0638
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step - accuracy: 1.0000 - loss: 0.0179
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0043
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 705ms/step - accuracy: 1.0000 - loss: 0.0014
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step - accuracy: 1.0000 - loss: 5.9128e-04
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 2.6656e-04
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 1.3

In [ ]:
# 5. Função para upload e classificação da imagem
def classificar_imagem(caminho_imagem):
  img = keras.preprocessing.image.load_img(caminho_imagem, target_size=(img_width, img_height))
  x = keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = keras.applications.mobilenet_v2.preprocess_input(x) # Pré-processamento do MobileNetV2
  predicoes = model.predict(x)
  classe = np.argmax(predicoes)
  return "Bebê" if classe == 0 else "Não Bebê"


In [ ]:
# 6. Upload da imagem no Colab
from google.colab import files
uploaded = files.upload()
nome_arquivo = list(uploaded.keys())[0]
caminho_arquivo = nome_arquivo

Saving IMG-20240523-WA0049.jpg to IMG-20240523-WA0049.jpg


In [ ]:
# 7. Classificação e exibição do resultado
resultado = classificar_imagem(caminho_arquivo)
print(f"A imagem é: {resultado}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
A imagem é: Não Bebê
